# CHAPTER 12
# Advanced pandas

## Categorical Data
- This section introduces the pandas **Categorical** type. 
- Using **categorical** data in statistics and machine learning applications.

### Background and Motivation
- Frequently, a column in a table may contain repeated instances of a smaller set of distinct values. 
- Functions like **unique** and **value_counts**, enable us to extract the distinct values from an array and compute their frequencies,nrespectively.

In [1]:
# Import libraries
import numpy as np
import pandas as pd

In [2]:
# Example Series
values = pd.Series(['apple', 'orange', 'apple', 'apple'] * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [3]:
# Select only the distinct values
pd.unique(values)

array(['apple', 'orange'], dtype=object)

- In data warehousing, a best practice is to use so called **dimension tables** containing the distinct values and storing the primary observations as **integer keys** referencing the dimension table.

In [4]:
# List of integers
values = pd.Series([0, 1, 0, 0] * 2)

# Unique list of strings
dim = pd.Series(['apple', 'orange'])

In [5]:
# We can use the take method to restore the original Series of strings
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

- This representation as integers is called the **categorical** or **dictionary-encoded** representation. 
- The array of distinct values can be called the **categories**, **dictionary**, or **levels of the data**. 
- We will use the terms **categorical** and categories. 
- The integer values that reference the categories are called the **category codes** or simply **codes**.
- The categorical representation can yield significant performance improvements when you are doing analytics. 
- You can also perform transformations on the categories while leaving the codes unmodified.

### Categorical Type in pandas
- **pandas** has a special **Categorical** type for holding data that uses the integer-based categorical representation or **encoding**.
- Unless explicitly specified, categorical conversions assume no specific ordering of the categories. 
- So the categories array may be in a different order depending on the ordering of the input data. 
- When using **from_codes** or any of the other constructors, you can indicate that the categories have a meaningful ordering.
- A categorical array can consist of any immutable value types.

In [6]:
# DataFrame example
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)

df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,14,0.013583
1,1,orange,14,1.876646
2,2,apple,12,2.108556
3,3,apple,4,1.896840
4,4,apple,12,3.103029
5,5,orange,10,0.350940
6,6,apple,5,0.013270
7,7,apple,9,1.147285


In [7]:
# We can convert df['fruit'] to categorical by calling the function astype
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [8]:
# The values for fruit_cat are not a NumPy array, but an instance of pandas.Categorical
c = fruit_cat.values
type(c)

pandas.core.arrays.categorical.Categorical

In [9]:
# The Categorical object has categories and codes attributes
c.categories

Index(['apple', 'orange'], dtype='object')

In [10]:
# The Categorical object has categories and codes attributes
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [11]:
# You can convert a DataFrame column to categorical by assigning the converted result:
df['fruit'] = df['fruit'].astype('category')
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [14]:
# You can also create pandas.Categorical directly from other types of Python sequences
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [15]:
# Use from_codes constructor if you have the codes & categories
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [16]:
# Ordered categories
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

### Computations with Categoricals
- Using **Categorical** in pandas compared with the non-encoded version (like an array of strings) generally behaves the same way. 
- Some parts of pandas, like the **groupby** function, perform better when working with categoricals.

In [17]:
# Create some random numeric data
np.random.seed(12345)
draws = np.random.randn(1000)
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [18]:
# Compute a quartile binning of this data
bins = pd.qcut(draws, 4)
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [19]:
# Use the labels argument to label the categories
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

['Q2', 'Q3', 'Q2', 'Q2', 'Q4', ..., 'Q3', 'Q2', 'Q1', 'Q3', 'Q4']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [20]:
# The labeled bins categorical does not contain information about the bin edges in the
# data, so we can use groupby to extract some summary statistics
bins = pd.Series(bins, name='quartile')
results = (pd.Series(draws).groupby(bins).agg(['count', 'min', 'max']).reset_index())
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


#### Better performance with categoricals
- If you do a lot of analytics on a particular dataset, converting to categorical can yield substantial overall performance gains. 
- A categorical version of a DataFrame column will often use significantly less memory, too.
- GroupBy operations can be significantly faster with categoricals because the underlying algorithms use the integer-based codes array instead of an array of strings.

In [20]:
# Let’s consider some Series with 10 million elements and a small number of distinct categories
N = 10000000
draws = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

#Now we convert labels to categorical
categories = labels.astype('category')

In [21]:
# Memory usage labels
labels.memory_usage()

80000128

In [22]:
# Memory usage categories
categories.memory_usage()

10000320

### Categorical Methods
- Series containing **categorical** data have several special methods similar to the **Series.str** specialized string methods. 
- This also provides convenient access to the **categories** and **codes**.

In [23]:
# Create an example of categories
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [24]:
# The special attribute cat provides access to categorical methods
# Get categories codes
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [25]:
# Get the categories names
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [26]:
# Use the set_categories method change the categories
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

In [27]:
# Use remove_unused_categories method to trim unobserved categories
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

**TABLE**: Categorical methods for Series in pandas

| Method                  | Description |
| :---                  |    :----    |
|add_categories| Append new (unused) categories at end of existing categories
|as_ordered| Make categories ordered
|as_unordered| Make categories unordered
|remove_categories| Remove categories, setting any removed values to null
|remove_unused_categories| Remove any category values which do not appear in the data
|rename_categories| Replace categories with indicated set of new category names; cannot change the number of categories
|reorder_categories| Behaves like rename_categories, but can also change the result to have ordered categories
|set_categories| Replace the categories with the indicated set of new categories; can add or remove categories

#### Creating dummy variables for modeling
- When you’re using **statistics** or **machine learning** tools, you’ll often transform categorical data into **dummy variables**, also known as **one-hot encoding**. 
- This involves creating a DataFrame with a column for each distinct category; these columns contain 1s for occurrences of a given category and 0 otherwise.

In [28]:
# Consider the previous example
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [29]:
# pandas.get_dummies function converts this one-dimensional categorical data into a DataFrame 
# containing the dummy variable
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## Advanced GroupBy Use
### Group Transforms and “Unwrapped” GroupBys
- There is another built-in method called **transform**, which is similar to apply but imposes more constraints on the kind of function you can use:
    - It can produce a scalar value to be broadcast to the shape of the group
    - It can produce an object of the same shape as the input group
    - It must not mutate its input

In [30]:
# Create a simple example
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [31]:
# group means by key
g = df.groupby('key').value
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [36]:
# Suppose instead we wanted to produce a Series of the same shape as df['value'] but
# with values replaced by the average grouped by 'key'
g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [32]:
# Equivalent with above example - works for built-in aggregation functions
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [33]:
# Compute the ranks in descending order for each group
g.transform(lambda x: x.rank(ascending=False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [39]:
# Consider a group transformation function composed from simple aggregations
def normalize(x):
    return (x - x.mean()) / x.std()

In [40]:
# We can obtain equivalent results in this case either using transform or apply
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

## The pipe Method
- Sometimes you need to use your own functions or functions from third-party libraries. 
- This is where the **pipe** method comes in.
- A potentially useful pattern for pipe is to generalize sequences of operations into **reusable functions**.

In [34]:
# Create example DataFrame
df = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data': np.random.randn(5)})
df

,key1,key2,data
0,a,one,-0.332027
1,a,two,0.977618
2,b,one,-0.344808
3,b,two,1.321220
4,a,one,-1.021145


In [35]:
# Let’s consider substracting the mean from a column
df['data'] = df.data - df.data.mean()
df

,key1,key2,data
0,a,one,-0.452198
1,a,two,0.857446
2,b,one,-0.464980
3,b,two,1.201048
4,a,one,-1.141316


In [36]:
# Suppose you wnat to do this to several columns
def group_demean(df, by, cols):
    result = df.copy()
    for c in cols:
        result[c] = df[c] - df[c].mean()
    return result

In [37]:
# Use pipe function to implement
result = df[df.data < 0].pipe(group_demean, ['key1', 'key2'], ['data'])
result

,key1,key2,data
0,a,one,0.233967
2,b,one,0.221185
4,a,one,-0.455152
